In [ ]:
#
# Project 2, starter code Part a
#

import math
import tensorflow as tf
import numpy as np
import pylab as plt
import pickle



NUM_CLASSES = 10
IMG_SIZE = 32
NUM_CHANNELS = 3
learning_rate = 0.001
epochs = 10
batch_size = 128

print_interval= 1


seed = 10
np.random.seed(seed)
tf.set_random_seed(seed)


trainX = None
trainY = None
testX = None
testY = None
x = None
y_ = None
logits = None
loss = None
train_step = None
W1 = None
b1 = None
conv_1 = None
pool_1 = None 
W2 = None
b2 = None
conv_2  = None
pool_2 = None 
W3 = None 
b3 = None

def load_data(file):
    with open(file, 'rb') as fo:
        try:
            samples = pickle.load(fo)
        except UnicodeDecodeError:  #python 3.x
            fo.seek(0)
            samples = pickle.load(fo, encoding='latin1')

    data, labels = samples['data'], samples['labels']

    data = np.array(data, dtype=np.float32)
    labels = np.array(labels, dtype=np.int32)

    
    labels_ = np.zeros([labels.shape[0], NUM_CLASSES])
    labels_[np.arange(labels.shape[0]), labels-1] = 1

    return data, labels_




def cnn(images):

    images = tf.reshape(images, [-1, IMG_SIZE, IMG_SIZE, NUM_CHANNELS])
    
    #Conv 1
    W1 = tf.Variable(tf.truncated_normal([9, 9, NUM_CHANNELS, 50], stddev=1.0/np.sqrt(NUM_CHANNELS*9*9)), name='weights_1')
    b1 = tf.Variable(tf.zeros([50]), name='biases_1')
    conv_1 = tf.nn.relu(tf.nn.conv2d(images, W1, [1, 1, 1, 1], padding='VALID') + b1)
    
    #pool 1
    pool_1 = tf.nn.max_pool(conv_1, ksize= [1, 2, 2, 1], strides= [1, 2, 2, 1], padding='VALID', name='pool_1')
    #dim = pool_1.get_shape()[1].value * pool_1.get_shape()[2].value * pool_1.get_shape()[3].value 
    
    #Conv 2
    W2 = tf.Variable(tf.truncated_normal([5, 5, 50, 60], stddev=1.0/np.sqrt(50*5*5)), name='weights_2')
    b2 = tf.Variable(tf.zeros([60]), name='biases_2')
    conv_2 = tf.nn.relu(tf.nn.conv2d(pool_1, W2, [1, 1, 1, 1], padding='VALID') + b2)
    
    #pool 2
    pool_2 = tf.nn.max_pool(conv_2, ksize= [1, 2, 2, 1], strides= [1, 2, 2, 1], padding='VALID', name='pool_2')
    dim = pool_2.get_shape()[1].value * pool_2.get_shape()[2].value * pool_2.get_shape()[3].value 
    #pool_2_flat = tf.reshape(pool_2, [-1, dim_1])
    
    #Fully connected layer 1 of size 300
    W_fc1 = tf.Variable(tf.truncated_normal([dim, 300], stddev=1.0/np.sqrt(dim)), name='weights_fc1')
    b_fc1 = tf.Variable(tf.zeros([300]), name='biases_fc1')
    h_pool2_flat = tf.reshape(pool_2, [-1, dim])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    dim_1 = 300
    
    #Softmax
    W3 = tf.Variable(tf.truncated_normal([dim_1, NUM_CLASSES], stddev=1.0/np.sqrt(dim_1)), name='weights_3')
    b3 = tf.Variable(tf.zeros([NUM_CLASSES]), name='biases_3')
    logits = tf.matmul(h_fc1, W3) + b3

    return W1, b1, conv_1, pool_1, W2, b2, conv_2, pool_2, W3, b3, logits


def main():
    global trainX, trainY, testX, testY, x, y_, logits, loss, train_step, W1, b1, conv_1, pool_1, W2, b2, conv_2, pool_2, W3, b3
    
    trainX, trainY = load_data('data_batch_1')
    print(trainX.shape, trainY.shape)
    
    testX, testY = load_data('test_batch_trim')
    print(testX.shape, testY.shape)

    trainX = (trainX - np.min(trainX, axis = 0))/np.max(trainX, axis = 0)

    # Create the model
    x = tf.placeholder(tf.float32, [None, IMG_SIZE*IMG_SIZE*NUM_CHANNELS])
    y_ = tf.placeholder(tf.float32, [None, NUM_CLASSES])

    
    W1, b1, conv_1, pool_1, W2, b2, conv_2, pool_2, W3, b3, logits = cnn(x)

    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=logits)
    loss = tf.reduce_mean(cross_entropy)

    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    return 
   

def batch_train():
    global trainX, trainY, testX, testY, x, y_, logits, loss, train_step, W1, b1, conv_1, pool_1, W2, b2, conv_2, pool_2, W3, b3

    main() 
    
    N = len(trainX)
    idx = np.arange(N)
    
    train_acc = []
    test_acc = []
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for e in range(epochs):
            np.random.shuffle(idx)
            trainX, trainY = trainX[idx], trainY[idx]

            #for start, end in zip(range(0, N, batch_size), range(batch_size, N, batch_size)):
                #_, loss_ = sess.run([train_step, loss], {x: trainX[start: end], y_: trainY[start: end]})
            _, loss_ = sess.run([train_step, loss], {x: trainX, y_: trainY})

            train_acc.append(sess.run(loss, feed_dict= {x: trainX, y_ : trainY}))
            test_acc.append(sess.run(loss, feed_dict= {x: testX, y_ : testY}))

            if e % print_interval == 0:
                print('epoch', e, 'entropy', loss_)

        return train_acc, test_acc
    
def printCombinedGraph(accuracies, epoch, hyperparam, y_label, label, saveAs = "def_combined.png"):
    n = len(hyperparam)

    fig, train_accuracy = plt.subplots(nrows=1, ncols=1) 
    for i in range(n):
        train_accuracy.plot(range(epoch), accuracies[i], label='{} = {}'.format(label, hyperparam[i]))
        

    plt.xlabel('epoch')
    train_accuracy.set_ylabel(y_label)
    train_accuracy.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.draw()
    plt.gcf().subplots_adjust(bottom=0.15)
    plt.savefig(saveAs, bbox_inches="tight")
    plt.show()

def plotTestPattern(X = -1):
    global trainX, trainY, testX, testY, x, y_, logits, loss, train_step, W1, b1, conv_1, pool_1, W2, b2, conv_2, pool_2, W3, b3

    #randomly showing an image
    if(X == -1):
        ind = np.random.randint(low=0, high=10000)
        X = trainX[ind,:]
    plt.figure()
    plt.gray()
    X_show = X.reshape(NUM_CHANNELS, IMG_SIZE, IMG_SIZE).transpose(1, 2, 0)
    plt.axis('off')
    plt.imshow(X_show)
    plt.savefig('./p1b_2.png')
    
    h_conv1_, h_pool1_, h_conv2_, h_pool2_ = sess.run([conv_1, pool_1, conv_2, pool_2], {x: X.reshape(1,784)})
    
def q1():
    global trainX, trainY, testX, testY, x, y_, logits, loss, train_step, W1, b1, conv_1, pool_1, W2, b2, conv_2, pool_2, W3, b3

    train_acc, test_acc = batch_train()
    
    printCombinedGraph([train_acc, test_acc], epochs, ["train accuracy", "test accuracy"], "accuracy", epochs, "q1_accuracies.png")
    
    print("test pattern 1")
    plotTestPattern()
    
    print("test pattern 2")
    plotTestPattern()
    
if __name__ == '__main__':
  q1()
